## Reading in data available through The Arabidopsis Information Resource (TAIR) database
The purpose of this notebook is to read in and do a preliminary survey of the data related to text and ontology annotations of text that is available through TAIR. The datasets need to be organized and also restructured into a standard format that will allow it be combined with datasets from other resources. This notebook takes the following input files that were downloaded from the TAIR and produces a set of files that have standard columns, which are listed and described below.

### Files read
```
plant-data/databases/tair/Locus_Germplasm_Phenotype_20190930.txt
plant-data/databases/tair/
plant-data/databases/tair/po_anatomy_gene_arabidopsis_tair.assoc
plant-data/databases/tair/po_temporal_gene_arabidopsis_tair.assoc
plant-data/databases/tair/ATH_GO_GOSLIM.txt
```

### Files created
```
plant-data/reshaped_data/tair_phenotype_descriptions.csv
plant-data/reshaped_data/tair_general_descriptions.csv
plant-data/reshaped_data/tair_all_go_annotations.csv
plant-data/reshaped_data/tair_curated_go_annotations.csv
plant-data/reshaped_data/tair_curated_po_annotations.csv
```

### Columns in the created files
* **species**: A string indicating what species the gene is in, currently uses the 3-letter codes from the KEGG database.
* **unique_gene_identifiers**: Pipe delimited list of gene identifers, names, models, etc which must uniquely refer to this gene.
* **other_gene_identifiers**: Pipe delimited list of other identifers, names, aliases, synonyms for the gene, which may but do not have to uniquely refer to it.
* **gene_models**: Pipe delimited list of gene model names that map to this gene.
* **descriptions**: A free text field for any descriptions of phenotyes associated with this gene.
* **annotations**: Pipe delimited list of gene ontology term identifiers.
* **sources**: Pipe delimited list of strings that indicate where this data comes from such as database names.

In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import sys
import os
import re
import warnings
import pandas as pd
import numpy as np
import itertools
import re
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

sys.path.append("../utils")
from constants import NCBI_TAG, EVIDENCE_CODES, ABBREVIATIONS_MAP

sys.path.append("../../oats")
from oats.nlp.preprocess import concatenate_with_delim, subtract_string_lists, replace_delimiter, concatenate_texts
from oats.nlp.small import remove_punctuation, remove_enclosing_brackets, add_prefix_safely

OUTPUT_DIR = "../reshaped_data"
mpl.rcParams["figure.dpi"] = 200
warnings.simplefilter('ignore')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
# Columns that should be in the final reshaped files.
reshaped_columns = ["species", 
 "unique_gene_identifiers", 
 "other_gene_identifiers", 
 "gene_models", 
 "descriptions", 
 "annotations", 
 "sources"]

# Creating and testing a lambda for finding gene model strings.
gene_model_pattern = re.compile("at[0-9]{1}g[0-9]+")
is_gene_model = lambda s: bool(gene_model_pattern.match(s.lower()))
assert is_gene_model("AT1G34534") == True
assert is_gene_model("at2g23452") == True
assert is_gene_model("ACAB1") == False

### File with genes and phenotype descriptions (Locus_Germplasm_Phenotype_20180702.txt)
Reading in the dataset of phenotypic descriptions. There is only one value specified as the gene name (locus name) in the original dataset so this column does not need to be parsed further. The descriptions commmonly use semi-colons to separate phrases. The next cell gets the distribution of the number of phrases in each description field for the dataset of text descriptions, as determined by a sentence parser. The majority of the descriptions are a single sentence or phrase, but some contain more.

In [3]:
filename = "../databases/tair/Locus_Germplasm_Phenotype_20190930.txt"
usecols = ["LOCUS_NAME", "PHENOTYPE"]
usenames = ["unique_gene_identifiers", "descriptions"]
renamed = {k:v for k,v in zip(usecols,usenames)}
df = pd.read_table(filename, usecols=usecols)
df.rename(columns=renamed, inplace=True)
df.dropna(axis="rows",inplace=True)
df.sample(20)

,unique_gene_identifiers,descriptions
15730,PDE245,pigment defective embryo; albino embryo.
4799,AT2G27150,"In the mutant, the ABA levels are reduced to a..."
4674,AT2G26330,"erecta phenotype, veins darker than interveina..."
1007,AT1G15550,Defective in gibberellin 3-oxidase 1; involved...
5558,AT2G39930,Reduced starch in leaves
12267,AT5G25350,Modest rescue of the ebf1-3 ebf2-3 phenotype. ...
9307,AT4G14730,Increased sensitivity to tunicamycin treatment.
11462,AT5G11710,defects in vacuolar trafficking of soluble car...
2644,AT1G63650,No trichomes; excessive root hair production. ...
762,AT1G10760,starch overproducer.


In [4]:
df.shape

(15442, 2)

In [6]:
# Plotting distributions of number of phrases in each description.
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.set_title("Phenotype Descriptions")
ax2.set_title("Phenotype Descriptions")
ax1.set_xlabel("Number of phrases")
ax2.set_xlabel("Number of words")
x1 = [len(sent_tokenize(x)) for x in df["descriptions"].values]
x2 = [len(word_tokenize(x)) for x in df["descriptions"].values]
ax1.hist(x1, bins=15, range=(0,15), density=False, alpha=0.8, histtype='stepfilled', color="black", edgecolor='none')
ax2.hist(x2, bins=30, range=(0,200), density=False, alpha=0.8, histtype='stepfilled', color="black", edgecolor='none')
fig.set_size_inches(15,4)
fig.tight_layout()
fig.show()
plt.close()

In [7]:
# Restructuring the dataset to include all expected column names.
df["species"] = "ath"
df["other_gene_identifiers"] = ""
df["gene_models"] = df["unique_gene_identifiers"].map(lambda x: "|".join([s for s in x.split("|") if is_gene_model(s)]))
df["annotations"] = ""
df["sources"] = "TAIR"
df = df[reshaped_columns]

# Outputting the dataset of phenotype descriptions to csv file.
path = os.path.join(OUTPUT_DIR,"tair_phenotype_descriptions.csv")
df.to_csv(path, index=False)
df.head(20)

,species,unique_gene_identifiers,other_gene_identifiers,gene_models,descriptions,annotations,sources
0,ath,ACL2,,,"flower stems are much reduced in length, semi-...",,TAIR
1,ath,ADE1,,,sustained and enhanced levels of ABA-regulated...,,TAIR
2,ath,ALB2,,,"white embryo and seedling (albino), lethal.",,TAIR
3,ath,ALR-104,,,incomplete penetrance; increased aluminum resi...,,TAIR
4,ath,ALR-128,,,incomplete penetrance; increased aluminum resi...,,TAIR
5,ath,ALS4,,,increased aluminum sensitivity; poor root grow...,,TAIR
6,ath,ALS5,,,incomplete penetrance; increased aluminum sens...,,TAIR
7,ath,ARC10,,,no visible whole plant phenotypic effect; mean...,,TAIR
8,ath,ARC2,,,mesophyll cells contain approximately half the...,,TAIR
9,ath,ARC4,,,fewer and larger chloroplasts than wild type i...,,TAIR


### File with curator summaries (Araport11_functional_descriptions_20190930.txt)

In [10]:
filename = "../databases/tair/Araport11_functional_descriptions_20190930.txt"
usecols = ["name", "Curator_summary"]
usenames = ["unique_gene_identifiers", "descriptions"]
renamed = {k:v for k,v in zip(usecols,usenames)}
df = pd.read_table(filename, usecols=usecols)
df.rename(columns=renamed, inplace=True)
df.dropna(axis="rows",inplace=True)
df.sample(20)

,unique_gene_identifiers,descriptions
1779,AT1G29900.1,Encodes carbamoyl phosphate synthetase (CPS) l...
29107,AT5G45050.2,Encodes a member of the WRKY Transcription Fac...
5170,AT2G21120.1,Encodes a putative magnesium transporter that ...
35154,AT4G10457.1,"Encodes a member of a family of small, secrete..."
18579,AT5G59340.1,Encodes a WUSCHEL-related homeobox gene family...
60676,AT2G43010.4,Isolated as a semidominant mutation defective ...
55297,AT1G26260.4,Encodes CIB5 (cryptochrome-interacting basic-h...
11699,AT3G47430.1,"member of the peroxin11 (PEX11) gene family, l..."
53105,AT1G55180.2,member of C2-PLD. subfamily Represents a phosp...
31182,AT1G03090.2,MCCA is the biotinylated subunit of the dimer ...


In [11]:
df.shape

(20630, 2)

In [15]:
# Restructuring the dataset to include all expected column names.
df["species"] = "ath"
df["other_gene_identifiers"] = ""
df["gene_models"] = df["unique_gene_identifiers"].map(lambda x: "|".join([s for s in x.split("|") if is_gene_model(s)]))
df["annotations"] = ""
df["sources"] = "TAIR"
df = df[reshaped_columns]

# Outputting the dataset of phenotype descriptions to csv file.
path = os.path.join(OUTPUT_DIR,"tair_general_descriptions.csv")
df.to_csv(path, index=False)
df.sample(20)

,species,unique_gene_identifiers,other_gene_identifiers,gene_models,descriptions,annotations,sources
10782,ath,AT3G26190.1,,AT3G26190.1,putative cytochrome P450,,TAIR
14933,ath,AT4G12720.1,,AT4G12720.1,Encodes a protein with ADP-ribose hydrolase ac...,,TAIR
742,ath,AT1G12910.1,,AT1G12910.1,"Encodes LIGHT-REGULATED WD1 (LWD1), a clock pr...",,TAIR
32449,ath,AT3G47810.3,,AT3G47810.3,Homolog of yeast retromer subunit VPS29. Part...,,TAIR
56480,ath,AT4G00450.2,,AT4G00450.2,Encodes the Arabidopsis homolog of the transcr...,,TAIR
7011,ath,AT2G24450.1,,AT2G24450.1,Fasciclin-like arabinogalactan protein. Possib...,,TAIR
41647,ath,AT5G10140.3,,AT5G10140.3,MADS-box protein encoded by FLOWERING LOCUS C ...,,TAIR
26546,ath,AT3G52380.1,,AT3G52380.1,Encodes a chloroplast RNA-binding protein that...,,TAIR
44583,ath,AT1G21630.4,,AT1G21630.4,TPLATE Adaptor complex subunit.,,TAIR
23325,ath,MUR7,,,a genetic locus involved in arabinose biosynth...,,TAIR


### File with gene ontology annotations (ATH_GO_GOSLIM.txt)
Read in the file containing names of loci and corresponding information relating to gene ontology term annotation. Not all of the columns are used here, only a subset of them are read in. The relationship column refers to the relationships between the gene for that loci and the term mentioned on that given line. Evidence refer to the method of acquiring and the confidence in the annotation itself. This is retained so that we can subset that dataset based on whether the annotations are experimentally confirmed or simply predicted annotations. This section also looks at how many unique values are present for each field.

Each term annotation in this dataset is also associated with an evidence code specifying the method by which this annotation was made, which is related to the confidence that we can have in this annotation, and the tasks that the annotation should be used for. About half of the term annotations were made computationally, but there are also a high number of annotations available from high confidence annotations such as experimentally validated, curator statements, and author statements.

In [9]:
filename = "../databases/tair/ATH_GO_GOSLIM.txt"
df_go = pd.read_table(filename, header=None, usecols=[0,2,3,4,5,9,12])
df_go.columns = ["locus","object","relationship","term_label","term_id","evidence_code","reference"]
unique_values = {col:len(pd.unique(df_go[col].values)) for col in df_go.columns}
print(df_go[["locus","object","term_id","evidence_code","reference"]].head(10))
print(df_go.shape)
for k,v in unique_values.items():
    print("{:18}{:8}".format(k,v))

       locus       object     term_id evidence_code                              reference
0  AT1G01010    AT1G01010  GO:0006355           IEA            AnalysisReference:501756966
1  AT1G01010    AT1G01010  GO:0006355           ISS    Publication:1345963|PMID:11118137  
2  AT1G01010    AT1G01010  GO:0044212           IPI  Publication:501786139|PMID:30356219  
3  AT1G01010    AT1G01010  GO:0005634           IEA            AnalysisReference:501756971
4  AT1G01010    AT1G01010  GO:0006355           IEA            AnalysisReference:501756966
5  AT1G01010    AT1G01010  GO:0006355           IEA            AnalysisReference:501756966
6  AT1G01010    AT1G01010  GO:0006355           IEA            AnalysisReference:501756966
7  AT1G01010  AT1G01010.1  GO:0005634           ISM            AnalysisReference:501780126
8  AT1G01010    AT1G01010  GO:0006355           ISS    Publication:1345963|PMID:11118137  
9  AT1G01010    AT1G01010  GO:0003700           ISS    Publication:1345963|PMID:11118137  

In [10]:
code_quantities = {c:len([x for x in df_go["evidence_code"] if EVIDENCE_CODES[x] in c]) 
             for c in list(set(EVIDENCE_CODES.values()))}
for k,v in code_quantities.items():
    print("{:25}{:8}".format(k,v))

high_throughput              5914
computational               47057
curator_statement           22809
experimental               131557
electronic_annotation       58110
author_statement            17092
phylogenetics               60627


In [11]:
# Restructuring the dataset to include all the expected column names.
df_go["species"] = "ath"
df_go["unique_gene_identifiers"] = df_go["locus"]
df_go["other_gene_identifiers"] = ""
df_go["gene_models"] = df_go["unique_gene_identifiers"].map(lambda x: "".join([s for s in x.split("|") if is_gene_model(s)]))
df_go["descriptions"] = ""
df_go["annotations"] = df_go["term_id"]
df_go["sources"] = "TAIR"
high_confidence_categories = ["experimental","author_statement","curator_statement"]
df_go["high_confidence"] = df_go["evidence_code"].apply(lambda x: EVIDENCE_CODES[x] in high_confidence_categories)

# Subset to only include the high-quality GO annotations and ouptut the dataset to a csv file.
df_go_high_confidence = df_go[df_go["high_confidence"]==True]
path = os.path.join(OUTPUT_DIR,"tair_curated_go_annotations.csv")
df_go_high_confidence = df_go_high_confidence[reshaped_columns]
df_go_high_confidence.to_csv(path, index=False)

# Outputting the dataset of annotations to a csv file.
df_go = df_go[reshaped_columns]
path = os.path.join(OUTPUT_DIR,"tair_all_go_annotations.csv")
df_go.to_csv(path, index=False)
df_go.head(20)

,species,unique_gene_identifiers,other_gene_identifiers,gene_models,descriptions,annotations,sources
0,ath,AT1G01010,,AT1G01010,,GO:0006355,TAIR
1,ath,AT1G01010,,AT1G01010,,GO:0006355,TAIR
2,ath,AT1G01010,,AT1G01010,,GO:0044212,TAIR
3,ath,AT1G01010,,AT1G01010,,GO:0005634,TAIR
4,ath,AT1G01010,,AT1G01010,,GO:0006355,TAIR
5,ath,AT1G01010,,AT1G01010,,GO:0006355,TAIR
6,ath,AT1G01010,,AT1G01010,,GO:0006355,TAIR
7,ath,AT1G01010,,AT1G01010,,GO:0005634,TAIR
8,ath,AT1G01010,,AT1G01010,,GO:0006355,TAIR
9,ath,AT1G01010,,AT1G01010,,GO:0003700,TAIR


### File with plant ontology term annotations (po_[termporal/anatomy]_gene_arabidopsis_tair.assoc)
There are two separate files available that include annotations of PO terms. The files do not have headers so column names are added based on how the columns are described in the accompanying available readme files. One of the files contains annotations for PO terms that are spatial, or describe a specific part of plant anatomy or plant molecular structures. The other file contains annotations for PO terms that are temporal, or refer to a specific process or stage of development. These files are each read in separately, and the next cells look at the quantity of unique values in the columns of each dataset. There are more spatial annotations than temporal annotations, and a greater number of terms used to describe the spatial annotations.

The next field combines the two datasets of PO annotations and looks at the number of unique values for each column in the resulting dataset. Because there is no overlap in the terms between the two, the datasets are simply appended to one another and the total unique terms are a sum of the individual datasets.

Each term annotation in this dataset is also associated with an evidence code specifying the method by which this annotation was made, which is related to the confidence that we can have in this annotation, and the tasks that the annotation should be used for. Almost all of the PO term annotations are high confidence, they are experimentally validated, and only a few of them are derived from author statements.

The strings which are described in the synonyms column are included as references to each gene, and are combined with the gene name mentioned in the symbol column into a single bar delimited list.

In [15]:
# Reading in the dataset of spatial PO term annotations.
filename = "../databases/tair/po_anatomy_gene_arabidopsis_tair.assoc"
df_po_spatial = pd.read_table(filename, header=None, skiprows=0, usecols=[2,4,5,6,9,10,11])
df_po_spatial.columns = ["symbol","term_id","references","evidence_code","name","synonyms","type"]
unique_values = {col:len(pd.unique(df_po_spatial[col].values)) for col in df_po_spatial.columns}
print(df_po_spatial.shape)
for k,v in unique_values.items():
    print("{:18}{:8}".format(k,v))

(371438, 7)
symbol               24085
term_id                369
references            2730
evidence_code            8
name                 25422
synonyms             28093
type                     6


In [17]:
# Reading in the dataset of temporal PO term annotations.
filename = "../databases/tair/po_temporal_gene_arabidopsis_tair.assoc"
df_po_temporal = pd.read_table(filename, header=None, skiprows=0, usecols=[2,4,5,6,9,10,11])
df_po_temporal.columns = ["symbol","term_id","references","evidence_code","name","synonyms","type"]
unique_values = {col:len(pd.unique(df_po_temporal[col].values)) for col in df_po_temporal.columns}
print(df_po_temporal.shape)
for k,v in unique_values.items():
    print("{:18}{:8}".format(k,v))

(186819, 7)
symbol               18006
term_id                 91
references             478
evidence_code            4
name                 18523
synonyms             18973
type                     5


In [18]:
# Looking at how many unique values each column has.
df_po = df_po_spatial.append(df_po_temporal, ignore_index=True)
unique_values = {col:len(pd.unique(df_po[col].values)) for col in df_po.columns}
print(df_po[["symbol","synonyms","evidence_code"]].head(10))
print(df_po.shape)
for k,v in unique_values.items():
    print("{:18}{:8}".format(k,v))

   symbol                                           synonyms evidence_code
0  ATARCA  AT1G18080|ATARCA|RACK1A_AT|RACK1A|SAC53|AtRACK...           IEP
1    HMG1  AT1G76490|HMG1|HMGR1|AtHMGR1|MAD3|hydroxy meth...           TAS
2    BAS1  AT2G26710|BAS1|CYP734A1|CYP72B1|PHYB ACTIVATIO...           TAS
3    BAK1  AT4G33430|BAK1|RKS10|SERK3|ELG|ATSERK3|ATBAK1|...           TAS
4    BRI1  AT4G39400|BRI1|CBB2|DWF2|BIN1|ATBRI1|BRASSINOS...           TAS
5    ATJ6    AT5G06910|ATJ6|J-domain protein 6|MOJ9.8|MOJ9_8           IEP
6    XPB2  AT5G41360|XPB2|ATXPB2|homolog of Xeroderma pig...           NAS
7    XPB1  AT5G41370|XPB1|ATXPB1|homolog of xeroderma pig...           NAS
8   PPCK2  AT3G04530|PPCK2|PEPCK2|ATPPCK2|phosphoenolpyru...           IEP
9    MYC2  AT1G32640|ATMYC2|RD22BP1|JAI1|JIN1|MYC2|ZBF1|J...           NAS
(558257, 7)
symbol               24148
term_id                460
references            2807
evidence_code            8
name                 25506
synonyms             28238
t

In [19]:
# Quantifying the number of annotations of each type.
code_quantities = {c:len([x for x in df_po["evidence_code"] if EVIDENCE_CODES[x] in c]) 
             for c in list(set(EVIDENCE_CODES.values()))}
for k,v in code_quantities.items():
    print("{:25}{:8}".format(k,v))

high_throughput              4527
computational                   0
curator_statement               0
experimental               553612
electronic_annotation           0
author_statement              118
phylogenetics                   0


In [20]:
df_po['name'] = df_po['name'].astype(str)
df_po.dtypes

symbol           object
term_id          object
references       object
evidence_code    object
name             object
synonyms         object
type             object
dtype: object

In [21]:
# Restructuring the dataset to include all the expected column names.
combine_columns = lambda row, columns: concatenate_with_delim("|", [row[column] for column in columns])
df_po["species"] = "ath"
df_po["unique_gene_identifiers"] = df_po.apply(lambda x: combine_columns(x, ["symbol", "name"]), axis=1)
df_po["other_gene_identifiers"] = df_po["synonyms"]
df_po["gene_model_strings_1"] =  df_po["unique_gene_identifiers"].map(lambda x: "|".join([s for s in x.split("|") if is_gene_model(s)]))
df_po["gene_model_strings_2"] =  df_po["unique_gene_identifiers"].map(lambda x: "|".join([s for s in x.split("|") if is_gene_model(s)]))
df_po["gene_models"] = df_po.apply(lambda x: combine_columns(x, ["gene_model_strings_1", "gene_model_strings_2"]), axis=1)
df_po["descriptions"] = ""
df_po["annotations"] = df_po["term_id"]
df_po["sources"] = "TAIR"
df_po["other_gene_identifiers"] = df_po.apply(lambda row: subtract_string_lists("|", row["other_gene_identifiers"],row["unique_gene_identifiers"]), axis=1)
df_po = df_po[reshaped_columns]

# Outputting the dataset of annotations to a csv file.
path = os.path.join(OUTPUT_DIR,"tair_curated_po_annotations.csv")
df_po.to_csv(path, index=False)
df_po.head(30)

KeyboardInterrupt: 